In [ ]:
"""
- 데이터 불러들이기
- 1. 사용데이터 : 05_생선_분류_데이터셋.csv
- 2. 독립변수 추출 : fish_input 변수
- 3. 종속변수 추출 : fish_target 변수
- 4. 훈련 및 테스트 데이터 분류 (75 : 25, 랜덤 42번)
- 5. 정규화하기 : train_scaled, test_scaled 변수
"""

In [2]:
import pandas as pd

file_path = "./data/05_생선_분류_데이터셋.csv"
fish =  pd.read_csv(file_path)

fish

### Species  Weight  Length  Diagonal  Height  Width
#    종류     무게    길이    대각선    높이    두께
# - 종속변수 : 종류 (범주형)
# - 독립변수 : 나머지 모두

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [3]:
### 독립변수 데이터 추출하기
# - 변수명 : fish_input
# - 훈련에 사용하는 array배열 타입으로 최종 결과 넣기
# - 데이터 행렬 차원 확인하기
fish_input = fish[["Weight", "Length", "Diagonal", "Height", "Width"]].to_numpy()
fish_input.shape

(159, 5)

In [4]:
### 종속변수 데이터 추출하기
# - 변수명 : fish_target
fish_target = fish["Species"].to_numpy()
fish_target.shape

(159,)

In [5]:
### 분리비율 75:25 (디폴트로 생략가능)
# - 랜덤값 : 42
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
              train_test_split(fish_input, fish_target,  
                               test_size=0.25, random_state=42)

In [6]:
### 독립변수 정규화하기
from sklearn.preprocessing import StandardScaler

In [7]:
### 클래스(객체) 생성
ss = StandardScaler()
ss

StandardScaler()

In [8]:
### 표준화를 위한 점수를 생성하기 위한 패턴 찾기
ss.fit(train_input)

StandardScaler()

In [9]:
### 찾은 패턴으로 데이터 변환하기
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [11]:
train_scaled.shape, test_scaled.shape

((119, 5), (40, 5))

# 로지스틱 회귀로 다중 분류 진행하기

In [12]:
### LogisticRegression 라이브러리 불러들이기
from sklearn.linear_model import LogisticRegression

In [13]:
### 모델(클래스) 생성하기
# - 다중분류 : 이진분류와 다르게 종속변수의 범주가 많음
# - 범주가 많다보니 -> 규제강도를 설정하는 것이 좋음
# - 범주가 많다는 것은 복잡도가 커진다는 의미
# - 복잡도가 커지는 경우 학습을 위한 훈련 반복 횟수가 많아짐
# - 반복을 통해 정확도를 높이게 됨

# - C : 규제강도(값이 작을수록 규제강도가 커짐, 기본값 1)
#     : 회귀모델의 릿지, 랏소의 alpha 값과 동일함(크면 강도가 커짐)
# - max_iter : 훈련반복 횟수(기본값 100, 반복횟수는 높여놓는 것이 좋음)
#            : 훈련 중 반복횟수를 초과하여 훈련이 필요할 경우 경고창 뜸
lr = LogisticRegression(C = 20, max_iter = 1000)
lr

LogisticRegression(C=20, max_iter=1000)

In [ ]:
# 이진분류시 시그모이드
# 다중분류시 소프트맥스 사용

In [14]:
### 훈련시키기
lr.fit(train_scaled, train_target)

LogisticRegression(C=20, max_iter=1000)

In [15]:
### 평가하기
train_score = lr.score(train_scaled, train_target) 
test_scroe = lr.score(test_scaled, test_target)

In [18]:
train_score, test_scroe

(0.9327731092436975, 0.925)

In [22]:
### 예측(predict)하기
# -예측값 : 테스트 데이터 5개만 추출하여 예측하기
lr.predict(test_scaled[ : 5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [25]:
### 각 범주별로 계산된 확률값 확인하기
import numpy as np
proba = lr.predict_proba(test_scaled[ : 5])
np.round(proba, decimals=3)

array([[0.   , 0.014, 0.841, 0.   , 0.136, 0.007, 0.003],
       [0.   , 0.003, 0.044, 0.   , 0.007, 0.946, 0.   ],
       [0.   , 0.   , 0.034, 0.935, 0.015, 0.016, 0.   ],
       [0.011, 0.034, 0.306, 0.007, 0.567, 0.   , 0.076],
       [0.   , 0.   , 0.904, 0.002, 0.089, 0.002, 0.001]])

In [26]:
lr.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

### 다중분류 알고리즘

In [27]:
### z값 확인하기
# - 각 리스트의 양수값 중 가장 큰값이 좋은 값
decision =  lr.decision_function(test_scaled[ : 5])
np.round(decision, decimals=3)

array([[ -6.499,   1.032,   5.163,  -2.727,   3.339,   0.326,  -0.634],
       [-10.86 ,   1.927,   4.77 ,  -2.396,   2.978,   7.841,  -4.259],
       [ -4.335,  -6.233,   3.174,   6.488,   2.357,   2.421,  -3.872],
       [ -0.684,   0.453,   2.647,  -1.185,   3.264,  -5.753,   1.258],
       [ -6.397,  -1.993,   5.815,  -0.109,   3.503,  -0.112,  -0.707]])

### 소프트맥스 함수(Softmax function)

In [ ]:
"""
- 소프트맥스 함수(알고리즘)은 다중분류에서 사용(딥러닝에서도 사용됨)
- 종속변수 2개 이상일 때 주로 사용됨
- 값의 범위 : 0~1 사이값 사용(0% ~ 100%)
- 범주(종속변수)들의 z값 전체를 확률로 변환하여 사용
- 종속변수들 중에 확률이 가장 큰 값을 선택하게 됨
"""

In [28]:
### 소프트맥스 알고리즘 불러들이기
from scipy.special import softmax

In [29]:
### 소프트맥스 결과값 확인하기
# - axis = 1 : 각 행의 열 단위로 확률 계산
proba = softmax(decision, axis = 1)
np.round(proba, decimals=3)

array([[0.   , 0.014, 0.841, 0.   , 0.136, 0.007, 0.003],
       [0.   , 0.003, 0.044, 0.   , 0.007, 0.946, 0.   ],
       [0.   , 0.   , 0.034, 0.935, 0.015, 0.016, 0.   ],
       [0.011, 0.034, 0.306, 0.007, 0.567, 0.   , 0.076],
       [0.   , 0.   , 0.904, 0.002, 0.089, 0.002, 0.001]])